In [ ]:
# Libs installation, import & setup
from pip._internal import main as pipmain
dict_libs = {i: i for i \
             in ["os", "pandas", "numpy", "glob", "calendar", "datetime", \
                 "time", "pyodbc", "tkinter", "splink"]}
"""
Libs used in this script:

os
> Provides a portable way of using operating system dependent functionality.
> https://docs.python.org/3/library/os.html

pandas
> Data analysis and manipulation tool.
> https://pandas.pydata.org/

numpy
> For scientific computing in Python & provides a multidimensional array object, 
  various derived objects (such as masked arrays and matrices), 
  & an assortment of routines for fast operations on arrays.
> https://numpy.org/doc/stable/index.html

glob
> Finds all the pathnames matching a specified pattern according 
  to the rules used by the Unix shell.
> https://docs.python.org/3/library/glob.html

calendar
> Allows to output calendars like the Unix `cal` program, 
  & provides additional useful functions related to the calendar.
> https://docs.python.org/3/library/calendar.html

datetime
> Supplies classes for manipulating dates and times.
> https://docs.python.org/3/library/datetime.html

time
> Provides various time-related functions.
> https://docs.python.org/3/library/time.html

pyodbc
> Open source Python module that makes accessing ODBC databases.
> https://github.com/mkleehammer/pyodbc

pyyaml
> YAML is a data serialization format designed for human readability and interaction 
  with scripting languages. PyYAML is a YAML parser and emitter for Python.
> https://pypi.org/project/PyYAML/

tkinter
> Interface to the Tcl/Tk GUI toolkit.
> https://docs.python.org/3/library/tkinter.html

splink
> For probabilistic record linkage (entity resolution) that allows deduplication 
  & linkage of records from datasets without unique identifiers.
> https://moj-analytical-services.github.io/splink/
"""

# Helper function to check libs & install them if not installed
def check_dependecies():
    missing_libs = dict()
    
    for lib_name in dict_libs.keys():
        try:
            __import__(lib_name, globals = globals())
            print("Lib imported successfully:", lib_name)
        except ImportError as import_error:
            print("Lib not present:", import_error)
            missing_libs.update({lib_name: dict_libs[lib_name]})

    libs_to_install = missing_libs.values()
    status = pipmain(["install", *libs_to_install])

    if status == 0:
        print("Missing lib installed successfully:", libs_to_install)    
        for lib_name in missing_libs.keys():
            __import__(lib_name, globals = globals())
            print("Lib imported successfully:", lib_name)
check_dependecies()
import os
import pandas as pd
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
import numpy as np
import glob
import calendar
from datetime import datetime, timedelta
import time
import pyodbc
from tkinter import simpledialog
from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl

# Helper function to connect to server
def connect_to_server(server):
    connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};' + \
                                'SERVER='+server+';' + \
                                'PORT=1433;'+
                                'Authentication=ActiveDirectoryInteractive;')
    
    return connection

# Init necessary vars
recon_month = simpledialog.askstring(title = "Enter month name", \
                                     prompt = f'FULL month name (e.g. January):{" " * 85}').capitalize()
start_date = datetime(datetime.now().year, \
                      list(calendar.month_name).index(recon_month.lower().capitalize()), \
                      1).date()
end_date = start_date.replace(day = 28) + timedelta(days = 5) \
           - timedelta(days = (start_date.replace(day = 28) + timedelta(days = 5)).day)
match_threshold = 0.7
index_donor_start, index_donor_end = 0, 0
index_donor_step = 10
index_donor_end += index_donor_step
# Create new folders to dump contain necessary files & file outputs
try:
    os.mkdir(f"./{recon_month} {str(start_date)[:4]} Recon")
    os.mkdir(f"./{recon_month} {str(start_date)[:4]} Recon/Templates")
    os.mkdir(f"./Reference Files")
except:
    pass
server = "HRICRM-PROD.CRM6.DYNAMICS.COM"
connection = connect_to_server(server) ; cursor = connection.cursor()

---
---
### Import Template Files Prep

In [ ]:
# Setup for donation statement files 
paths = glob.glob(f"./{recon_month} {str(start_date)[:4]} Recon/*")
list_platforms = ["braintree", "benevity", "goodcompany", "good2give", "ctuk", "insight"]
available_platform_statements = list(set([i for i in list_platforms for j in paths if i in j]))
available_dfs = []

# CRM Codes
df_CRM_codes = \
pd.read_excel(f"./Reference Files/FRO_CRM_and_Finance-Activity_and_Coding_Reference.xlsb")

# BrainTree
if "braintree" in available_platform_statements:
    df_BrainTree = pd.concat((pd.read_csv(path) for path in paths if (("braintree" in path) & ("transaction_search" in path))), 
                             axis = 0).reset_index(drop = True)
    df_BrainTree["Donor"] = df_BrainTree["Customer First Name"].str.capitalize() + " " + \
                            df_BrainTree["Customer Last Name"].str.capitalize()
    df_BrainTree["Donor"] = df_BrainTree["Donor"].str.title()
    df_BrainTree["Payment Method"] = df_BrainTree["Card Type"] + " - " + df_BrainTree["Last Four of Credit Card"].astype("str")
    df_BrainTree = df_BrainTree[["Settlement Date", "Transaction ID", "Subscription ID", "Amount Submitted For Settlement", \
                                 "Currency ISO Code", "Campaign Id", \
                                 "Donor", "Customer First Name", "Customer Last Name", \
                                 "Customer Email", "Customer Phone", "Payment Method", "Card Type"]]
    df_BrainTree["Originating Campaign"] = \
    np.select([(df_BrainTree["Subscription ID"].isnull()) & (df_BrainTree["Currency ISO Code"] == "AUD"), \
               (df_BrainTree["Subscription ID"].isnull()) & (df_BrainTree["Currency ISO Code"] == "NZD"), \
               (df_BrainTree["Subscription ID"].isnull()) & (df_BrainTree["Currency ISO Code"] == "GBP"), \
               (df_BrainTree["Subscription ID"].notna()) & (df_BrainTree["Currency ISO Code"] == "AUD"), \
               (df_BrainTree["Subscription ID"].notna()) & (df_BrainTree["Currency ISO Code"] == "NZD"), \
               (df_BrainTree["Subscription ID"].notna()) & (df_BrainTree["Currency ISO Code"] == "GBP")], \
              ["AUSG-DGAQ-HR", "NZSG-DGAQ-HR", "UKSG-DGAQ-HR", "AURG-DGAQ-HR", "NZRG-DGAQ-HR", "UKRG-DGAQ-HR"])
    df_BrainTree["Originating Campaign"] = \
    np.select([df_BrainTree["Campaign Id"].str.contains("tax", na = False), \
               df_BrainTree["Campaign Id"].str.contains("xmas", na = False)], \
              ["AUSG-DMRN-HR-" + df_BrainTree["Campaign Id"].str[-2:] + "02-D1", \
               "AUSG-DMRN-HR-" + df_BrainTree["Campaign Id"].str[-2:] + "04-W2"], \
              default = df_BrainTree["Originating Campaign"])
    df_BrainTree["Primary Designation"] = \
    [df_CRM_codes[df_CRM_codes["CampaignID"] == df_BrainTree["Originating Campaign"][i]]["DesignationID (Finance GL Codes)"] \
     .values[0].split()[0] \
     for i in range(len(df_BrainTree["Originating Campaign"]))]
    df_BrainTree["Appeal"] = \
    [df_CRM_codes[df_CRM_codes["CampaignID"] == df_BrainTree["Originating Campaign"][i]]["AppealID"].values[0] \
     for i in range(len(df_BrainTree["Originating Campaign"]))]
    df_BrainTree["Appeal"] = [i[0] + " " + i[1] for i in df_BrainTree["Appeal"].str.split()]
    df_BrainTree["Payment Type"] = "External Credit/Debit"
    df_BrainTree["Settlement Date"] = pd.to_datetime(df_BrainTree["Settlement Date"], yearfirst = True).dt.date
    df_BrainTree[["Data Entry Source", "Package"]] = "Online", "Online"
    df_BrainTree["Data Entry Reference"] = \
    np.select([(df_BrainTree["Currency ISO Code"] == "AUD") & (df_BrainTree["Originating Campaign"] == "AUSG-DGAQ-HR"), \
               (df_BrainTree["Currency ISO Code"] == "NZD") & (df_BrainTree["Originating Campaign"] == "NZSG-DGAQ-HR"), \
               (df_BrainTree["Currency ISO Code"] == "GBP") & (df_BrainTree["Originating Campaign"] == "UKSG-DGAQ-HR"), \
               (df_BrainTree["Currency ISO Code"] == "AUD") & (df_BrainTree["Originating Campaign"] == "AURG-DGAQ-HR"), \
               (df_BrainTree["Currency ISO Code"] == "NZD") & (df_BrainTree["Originating Campaign"] == "NZRG-DGAQ-HR"), \
               (df_BrainTree["Currency ISO Code"] == "GBP") & (df_BrainTree["Originating Campaign"] == "UKRG-DGAQ-HR"), \
               (df_BrainTree["Currency ISO Code"] == "AUD") & (df_BrainTree["Originating Campaign"] == "AUSG-DMRN-HR-" + df_BrainTree["Campaign Id"].str[-2:] + "02-D1"), \
               (df_BrainTree["Currency ISO Code"] == "AUD") & (df_BrainTree["Originating Campaign"] == "AUSG-DMRN-HR-" + df_BrainTree["Campaign Id"].str[-2:] + "04-W2")],
              [f'Braintree AU Online OO {str(start_date).replace("-", "")}_{str(end_date).replace("-", "")}', \
               f'Braintree NZ Online OO {str(start_date).replace("-", "")}_{str(end_date).replace("-", "")}', \
               f'Braintree UK Online OO {str(start_date).replace("-", "")}_{str(end_date).replace("-", "")}', \
               f'Braintree AU Online RG {str(start_date).replace("-", "")}_{str(end_date).replace("-", "")}', \
               f'Braintree NZ Online RG {str(start_date).replace("-", "")}_{str(end_date).replace("-", "")}', \
               f'Braintree UK Online RG {str(start_date).replace("-", "")}_{str(end_date).replace("-", "")}', \
               f'Braintree AU Online OO Tax Appeal {str(start_date).replace("-", "")}_{str(end_date).replace("-", "")}', \
               f'Braintree AU Online OO Xmas Appeal {str(start_date).replace("-", "")}_{str(end_date).replace("-", "")}'])
    df_BrainTree["Data Entry Reference"] = np.select([df_BrainTree["Amount Submitted For Settlement"] < 0], \
                                                     [df_BrainTree["Data Entry Reference"].astype("str") + " Refund"], \
                                                     default = df_BrainTree["Data Entry Reference"])
    df_BrainTree["Identifier"] = "Braintree - " + df_BrainTree["Transaction ID"]
    df_BrainTree = df_BrainTree.rename(columns = {"Settlement Date": "Book Date", "Amount Submitted For Settlement": "Amount", \
                                                  "Currency ISO Code": "Currency", "Customer Email": "Email", \
                                                  "Customer First Name": "First Name", "Customer Last Name": "Last Name", \
                                                  "Customer Phone": "Phone", "Card Type": "Card Brand", \
                                                  "Campaign Id": "donation_type"})
    available_dfs.append(df_BrainTree)

# Good2Give
if "good2give" in available_platform_statements:
    df_Good2Give = pd.concat((pd.read_csv(path, skiprows = 2, skipfooter = 2, engine = "python") \
                              for path in paths if (("good2give" in path) & ("heart research institute" in path))), 
                             axis = 0).reset_index(drop = True)
    df_Good2Give = df_Good2Give.dropna(subset = ["Donation Confirmation Number"])
    # 5% accosted to admin fees
    df_Good2Give["Donation Amount"] = 0.95 * \
                                      df_Good2Give["Donation Amount"].str.replace("$", "", regex = False).astype("float")
    df_Good2Give = df_Good2Give[["Good2Give Donor Id", "Donor Firstname", "Donor Lastname", "Donation Amount", \
                                 "Employer Name", "Donor Email", "Donation Type", "Donation Confirmation Number"]]
    df_Good2Give["Donor"] = \
    df_Good2Give["Donor Firstname"].str.capitalize() + " " + df_Good2Give["Donor Lastname"].str.capitalize()
    df_Good2Give["Donor"] = df_Good2Give["Donor"].str.title()
    df_Good2Give[["Currency", "Originating Campaign", "Primary Designation", "Appeal"]] = \
    "AUD", "AUMG-COAQ-HR-WP", "FA-PHWG-1001", "PHIL CORP"
    df_Good2Give["Identifier"] = "Good2Give - " + df_Good2Give["Donation Confirmation Number"].astype("str")
    df_Good2Give["Payment Type"] = "Other"
    df_Good2Give["Data Entry Source"] = "Direct Deposit"
    df_Good2Give["Ref"] = \
    np.select([df_Good2Give["Donation Type"] == "Workplace Giving", \
               (df_Good2Give["Donation Type"] == "Employer Matched") | (df_Good2Give["Donation Type"] == "Match")], \
              ["Good2Give Donor ID " + df_Good2Give["Good2Give Donor Id"].astype("int").astype("str"), \
               "Good2Give Donor ID " + df_Good2Give["Good2Give Donor Id"].astype("int").astype("str") + " - Employer Matched"])
    df_Good2Give["Data Entry Reference"] = f'Good2Give {str(start_date).replace("-", "")}_{str(end_date).replace("-", "")}'
    df_Good2Give = df_Good2Give.rename(columns = {"Donor Firstname": "First Name", "Donor Lastname": "Last Name", \
                                                  "Donation Amount": "Amount", "Donor Email": "Email", \
                                                  "Donation Confirmation Number": "Transaction ID", \
                                                  "Employer Name": "Organization Name", \
                                                  "Donation Type": "donation_type"}) \
                               .drop(labels = ["Good2Give Donor Id"], axis = 1)
    available_dfs.append(df_Good2Give)

# Benevity
if "benevity" in available_platform_statements:
    df_Benevity = pd.concat((pd.read_csv(path, skiprows = 11, skipfooter = 4, engine = "python") \
                             for path in paths if ("benevity" in path)), 
                            axis = 0).reset_index(drop = True)
    # Sum up below 2 columns to match amt received in NABGEN
    df_Benevity["Total Donation"] = \
    df_Benevity["Total Donation to be Acknowledged"].str.replace(",", "").astype("float") + df_Benevity["Match Amount"]
    df_Benevity = df_Benevity[["Company", "Donor First Name", "Donor Last Name", "Email", \
                               "Address", "City", "State/Province", "Postal Code", "Match Amount", \
                               "Total Donation", "Transaction ID", "Donation Frequency", "Currency"]]
    df_Benevity["Donor"] = \
    df_Benevity["Donor First Name"].str.capitalize() + " " + df_Benevity["Donor Last Name"].str.capitalize()
    df_Benevity["Donor"] = df_Benevity["Donor"].str.title()
    df_Benevity["Donor"] = np.select([df_Benevity["Donor"].str.contains("Not shared")], \
                                     [df_Benevity["Company"].astype("str") + " (Anonymous Employee)"], \
                                     default = df_Benevity["Donor"])
    for i in ["Donor First Name", "Donor Last Name", "Address", "City", "State/Province", "Postal Code"]:
        df_Benevity[i] = df_Benevity[i].astype("str")
        df_Benevity[i] = [df_Benevity[i][j].replace(df_Benevity[i][j], "") if "Not shared" \
                          in df_Benevity[i][j] else df_Benevity[i][j] for j in range(len(df_Benevity[i]))]
    df_Benevity["Identifier"] = "Benevity - " + df_Benevity["Transaction ID"].astype("str")
    df_Benevity[["Payment Type", "Package", "Data Entry Source"]] = "Other", "Benevity", "Direct Deposit"
    df_Benevity["Originating Campaign"] = np.select([df_Benevity["Donation Frequency"] == "One Time", \
                                                     df_Benevity["Donation Frequency"] == "Recurring"], \
                                                    ["AUSG-DGAQ-CM", "AUMG-COAQ-HR-WP"])
    df_Benevity["Primary Designation"] = np.select([df_Benevity["Donation Frequency"] == "One Time", \
                                                    df_Benevity["Donation Frequency"] == "Recurring"], \
                                                   ["FA-SGCF-1001", "FA-PHWG-1001"])
    df_Benevity["Appeal"] = np.select([df_Benevity["Donation Frequency"] == "One Time", \
                                       df_Benevity["Donation Frequency"] == "Recurring"], \
                                      ["SG DIGI", "PHIL CORP"])
    df_Benevity["Ref"] = \
    np.select([df_Benevity["Match Amount"] == 0, df_Benevity["Match Amount"] != 0], \
              ["AUSOGF", \
               "AUSOGF - " + df_Benevity["Company"].astype("str") + " " + df_Benevity["Currency"].astype("str") + " " + df_Benevity["Match Amount"].astype("str") + " Matched Gift"])
    df_Benevity["Data Entry Reference"] = f'AUSOGF {str(start_date).replace("-", "")}_{str(end_date).replace("-", "")}'
    df_Benevity["Data Entry Source"] = "Direct Deposit"
    df_Benevity = df_Benevity.rename(columns = {"Company": "Organization Name", \
                                                "Donor First Name": "First Name", "Donor Last Name": "Last Name", \
                                                "Address": "Street 1", "Postal Code": "ZIP/Postal Code", \
                                                "Total Donation": "Amount", \
                                                "Donation Frequency": "donation_type"}) \
                             .drop(labels = ["Match Amount"], axis = 1)
    available_dfs.append(df_Benevity)

# GoodCompany
if "goodcompany" in available_platform_statements:
    df_GoodCompany = pd.concat((pd.read_csv(path) for path in paths if ("goodcompany" in path)), 
                               axis = 0).reset_index(drop = True)
    df_GoodCompany["Donor"] = df_GoodCompany["First Name"].str.capitalize() + " " + \
                              df_GoodCompany["Last Name"].str.capitalize()
    df_GoodCompany["Donor"] = df_GoodCompany["Donor"].str.title()
    df_GoodCompany["State"] = df_GoodCompany["State"].str.upper()
    df_GoodCompany = df_GoodCompany[["Date", "Amount", "Site", "Type", "Recurrence", \
                                     "Donor", "First Name", "Last Name", "Email", "State", "Postal Address", "Suburb", "Post Code", \
                                     "Transaction Reference"]]
    df_GoodCompany["Originating Campaign"] = np.select([(df_GoodCompany["Type"] == "Payroll Giving") | (df_GoodCompany["Type"] == "Matched Giving")], \
                                                       ["AUMG-COAQ-HR-WP"], default = "AUSG-DGAQ-CM")
    df_GoodCompany["Primary Designation"] = np.select([(df_GoodCompany["Type"] == "Payroll Giving") | (df_GoodCompany["Type"] == "Matched Giving")], \
                                                       ["FA-PHWG-1001"], default = "FA-SGCF-1001")
    df_GoodCompany["Appeal"] = np.select([(df_GoodCompany["Type"] == "Payroll Giving") | (df_GoodCompany["Type"] == "Matched Giving")], \
                                         ["PHIL CORP"], default = "SG DIGI")
    df_GoodCompany["Date"] = pd.to_datetime(df_GoodCompany["Date"], yearfirst = True).dt.date
    df_GoodCompany["Identifier"] = "GoodCompany - " + df_GoodCompany["Transaction Reference"].astype("str")
    # df_GoodCompany["Donor"] = np.select([df_GoodCompany["Donor"].isnull()], \
    #                                     [df_GoodCompany["Site"].astype("str") + " (Anonymous Employee)"], \
    #                                     default = df_GoodCompany["Donor"])
    df_GoodCompany[["Payment Type", "Package", "Data Entry Source", "Currency"]] = "Other", "GoodCompany/Karma Currency", "Direct Deposit", "AUD"
    df_GoodCompany["Data Entry Reference"] = f'GoodCompany Fd {str(start_date).replace("-", "")}_{str(end_date).replace("-", "")}'
    df_GoodCompany["Ref"] = df_GoodCompany["Transaction Reference"]
    df_GoodCompany = df_GoodCompany.rename(columns = {"Date": "Book Date", "State": "State/Province", "Site": "Organization Name", \
                                                      "Postal Address": "Street 1", "Suburb": "City", "Post Code": "ZIP/Postal Code", \
                                                      "Transaction Reference": "Transaction ID", \
                                                      "Recurrence": "donation_type"})
    available_dfs.append(df_GoodCompany)

# Charities Trust UK
if "ctuk" in available_platform_statements:
    df_CTUK = pd.concat((pd.read_csv(path) for path in paths if ("ctuk" in path)), 
                        axis = 0).reset_index(drop = True)
    df_CTUK["Donor"] = df_CTUK["First Name"].str.capitalize() + " " + df_CTUK["Last Name"].str.capitalize()
    df_CTUK["Donor"] = df_CTUK["Donor"].str.title()
    df_CTUK["Identifier"] = "Charities Trust UK - " + df_CTUK["PGA Donor Ref Number"].astype("str")
    df_CTUK = df_CTUK[["PGA Donor Ref Number", "Employer Name", "First Name", "Last Name", "Donor", "Donor Email", "Donor Tel No", \
                       "Address 1", "Address 2", "Address 3", "Postcode", \
                       "Total Donation to Charity", "Donor Preference Post", "Donor Preference Email", \
                       "Donor Preference Phone", "Donor Preference SMS", "Identifier"]]
    df_CTUK[["Currency", "Originating Campaign", "Primary Designation", "Appeal", "Package"]] = \
    "GBP", "UKMG-COAQ-HR-WP", "FG-PHWG-1001", "PHIL CORP", "Charities Trust UK"
    df_CTUK["Payment Type"] = "Other"
    df_CTUK["Data Entry Source"] = "Direct Deposit"
    df_CTUK["Data Entry Reference"] = f'Charities Trust UK {str(start_date).replace("-", "")}_{str(end_date).replace("-", "")}'
    df_CTUK["Ref"] = "Charities Trust UK " + df_CTUK["PGA Donor Ref Number"].astype("str")
    df_CTUK["donation_type"] = "Recurring"
    df_CTUK = df_CTUK.rename(columns = {"PGA Donor Ref Number": "Transaction ID", "Employer Name": "Organization Name", \
                                        "Address 1": "Street 1", "Address 2": "Street 2", "Address 3": "Street 3", "Postcode": "ZIP/Postal Code", \
                                        "Donor Tel No": "Phone", \
                                        "Total Donation to Charity": "Amount", \
                                        "Donor Email": "Email"})
    available_dfs.append(df_CTUK)
    
# Insight
if "insight" in available_platform_statements:
    df_Insight = pd.concat((pd.read_excel(path) for path in paths if ("insight" in path)), 
                           axis = 0).reset_index(drop = True)
    df_Insight = df_Insight[["Week Ending", "Bank Date", "Insight WP Number", "Customer Id", "First name", "SurName", "Address1", "Suburb", "State", \
                             "Post Code", "Telephone", "Email Address", "Donation Value $", "Sale Type"]]
    df_Insight["Donor"] = df_Insight["First name"].str.capitalize() + " " + df_Insight["SurName"].str.capitalize()
    df_Insight["Donor"] = df_Insight["Donor"].str.title()
    df_Insight[["Data Entry Source", "Originating Campaign", "Primary Designation", "Appeal", "Package", "Payment Type", "Ref", "Country/Region", "Currency"]] = \
    "TM", "AUSG-TMAQ-IN-DONATE", "FA-SGLR-1001", "SG LOTT/RAFF", "DONATIONS", "Other", "Receipted by Insight", "Australia", "AUD"
    df_Insight["Data Entry Reference"] = "Westpac_Insight OO " + df_Insight["Week Ending"].astype("str").str.replace("-", "")
    df_Insight["Identifier"] = "Insight - " + df_Insight["Customer Id"]
    df_Insight["Post Code"] = np.select([df_Insight["Post Code"].astype("str").str.len() < 4], ["0" + df_Insight["Post Code"].astype("str")], \
                                        default = df_Insight["Post Code"].astype("str"))
    df_Insight = df_Insight.rename(columns = {"Bank Date": "Book Date", "Insight WP Number": "Related Constituent", \
                                              "First name": "First Name", "SurName": "Last Name", "Address1": "Street 1", "Suburb": "City", \
                                              "State": "State/Province", "Post Code": "ZIP/Postal Code", "Telephone": "Phone", "Email Address": "Email", \
                                              "Donation Value $": "Amount", "Sale Type": "SG External Program Paymentmethod"}) \
                           .drop(labels = ["Customer Id"], axis = 1)
    available_dfs.append(df_Insight)    

# Final df Transaction template
df_transaction = pd.read_excel("./Reference Files/Transaction Template Official.xlsx")
list_df_transaction_columns = list(df_transaction.columns)
available_dfs.insert(0, df_transaction)
df_transaction = pd.concat(available_dfs, axis = 0).iloc[1:, :].reset_index(drop = True)
df_transaction[["Status Reason", "Status"]] = "Completed", "Active"
df_transaction["Status Reason"] = np.select([df_transaction["Amount"] < 0], ["Refund"], default = df_transaction["Status Reason"])
df_transaction["Anonymous"] = np.select([df_transaction["Donor"].isnull()], 
                                        ["Yes"], default = "No")
df_transaction["Country/Region"] = \
np.select([df_transaction["Currency"] == "AUD", \
           df_transaction["Currency"] == "NZD", \
           df_transaction["Currency"] == "GBP"], \
          ["Australia", "New Zealand", "United Kingdom"], default = df_transaction["Country/Region"])
df_transaction["Type"] = "Donation"
df_transaction["Phone"] = df_transaction["Phone"].str.replace(" ", "")
df_transaction["Phone"] = np.select([df_transaction["Phone"].notna()], ["0" + df_transaction["Phone"].astype("Int64").astype("str")], \
                                    default = df_transaction["Phone"])
df_transaction["ZIP/Postal Code"] = pd.to_numeric(df_transaction["ZIP/Postal Code"], errors = "coerce")
df_transaction["ZIP/Postal Code"] = np.select([df_transaction["ZIP/Postal Code"].astype("Int64").astype("str").str.len() < 4, df_transaction["ZIP/Postal Code"].isnull()], \
                                              ["0" + df_transaction["ZIP/Postal Code"].astype("Int64").astype("str"), None], \
                                              default = df_transaction["ZIP/Postal Code"].astype("Int64").astype("str"))
df_transaction["Email"] = df_transaction["Email"].str.lower()
# Anonymous handling unprocessed in previous steps
list_columns = ["Primary Designation", "Originating Campaign", "Appeal", "Donor", \
                "First Name", "Last Name"]
list_choices = [["FA-SGDM-1990", "FN-SGDM-1990", "FG-SGDM-1990"], \
                ["AUSG-DMAQ-UN", "NZSG-DMAQ-UN", "UKSG-DMAQ-UN"], \
                ["SG GENERAL", "SG GENERAL", "SG GENERAL"], \
                ["Anonymous AU", "Anonymous NZ", "Anonymous UK"], \
                ["Anonymous AU", "Anonymous NZ", "Anonymous UK"], \
                ["Anonymous AU", "Anonymous NZ", "Anonymous UK"]]
for i in range(len(list_columns)):
    choice_1, choice_2, choice_3 = list_choices[i][0], list_choices[i][1], list_choices[i][2],
    df_transaction[list_columns[i]] = \
    np.select([((df_transaction["Anonymous"] == "Yes") & (df_transaction["Currency"] == "AUD") \
                & (df_transaction["Donor"].isnull()) & (df_transaction["Primary Designation"].isnull()) \
                & (df_transaction["Originating Campaign"].isnull()) & (df_transaction["Appeal"].isnull())),          
               ((df_transaction["Anonymous"] == "Yes") & (df_transaction["Currency"] == "NZD") \
                & (df_transaction["Donor"].isnull()) & (df_transaction["Primary Designation"].isnull()) \
                & (df_transaction["Originating Campaign"].isnull()) & (df_transaction["Appeal"].isnull())),       
               ((df_transaction["Anonymous"] == "Yes") & (df_transaction["Currency"] == "GBP") \
                & (df_transaction["Donor"].isnull()) & (df_transaction["Primary Designation"].isnull()) \
                & (df_transaction["Originating Campaign"].isnull()) & (df_transaction["Appeal"].isnull()))], \
              [choice_1, choice_2, choice_3], default = df_transaction[list_columns[i]])
df_transaction["Donor"] = np.select([((df_transaction["Anonymous"] == "Yes") & (df_transaction["Donor"].isnull()) \
                                      & (df_transaction["Currency"] == "AUD")), \
                                     ((df_transaction["Anonymous"] == "Yes") & (df_transaction["Donor"].isnull()) \
                                      & (df_transaction["Currency"] == "NZD")), \
                                     ((df_transaction["Anonymous"] == "Yes") & (df_transaction["Donor"].isnull()) \
                                      & (df_transaction["Currency"] == "GBP"))], \
                                    ["Anonymous AU", "Anonymous NZ", "Anonymous UK"], default = df_transaction["Donor"])

# Get Payment Schedule ID
# While loop to re-establish connection to server in case it fails/errors out until connection is re-established
while True:
    try:
        query_result = \
        cursor.execute("""
                       SELECT msnfp_customeridname, msnfp_customerid,
                              msnfp_emailaddress1, msnfp_name, statuscode, 
                              msnfp_originatingcampaignidname,
                              msnfp_amount_receipted
                       FROM dbo.msnfp_paymentschedule
                       WHERE msnfp_name LIKE '%Braintree%'
                             AND statuscode <> '844060001'
                       """).fetchall()
    except pyodbc.Error as error:
        if error.args[0] == "08S01": 
            connection.close()
            connection = connect_to_server(server)
            cursor = connection.cursor()
        try:
            query_result = \
            cursor.execute("""
                           SELECT msnfp_customeridname, msnfp_customerid,
                                  msnfp_emailaddress1, msnfp_name, statuscode, 
                                  msnfp_originatingcampaignidname,
                                  msnfp_amount_receipted
                           FROM dbo.msnfp_paymentschedule
                           WHERE msnfp_name LIKE '%Braintree%'
                                 AND statuscode <> '844060001'
                           """).fetchall()
        except: pass
    if query_result != np.nan: break
df_transaction["join_on_payschedID"] = \
np.select([(df_transaction["Data Entry Reference"].str.contains("Braintree")) & (df_transaction["Data Entry Reference"].str.contains("RG"))], \
          [df_transaction["Subscription ID"]], \
          default = None)
# Store payschedID query result to ensure newly assigned payschedIDs are unique 
df_payschedID = pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description])
df_payschedID["join_on_payschedID"] = \
np.select([df_payschedID["msnfp_name"].str.contains("DIGI")], \
          [df_payschedID["msnfp_name"].str.split(" ").str[-1]], \
          default = None)
df_transaction = df_transaction.merge(df_payschedID, on = ["join_on_payschedID"], how = "left") \
                               .drop(labels = ["msnfp_customeridname", "msnfp_customerid", "msnfp_emailaddress1", "statuscode", \
                                               "msnfp_originatingcampaignidname", "msnfp_amount_receipted"], \
                                     axis = 1)
df_transaction = \
pd.concat([df_transaction[df_transaction["Donor"].isin(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description]) \
                                                       [(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description])["msnfp_customeridname"].str.lower() \
                                                        .isin(df_transaction[df_transaction["Payment Schedule"].isnull()]["Donor"].str.lower()))]["msnfp_customeridname"])] \
           .merge(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description]) \
                  [(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description])["msnfp_customeridname"].str.lower() \
                    .isin(df_transaction[df_transaction["Payment Schedule"].isnull()]["Donor"].str.lower()))][["msnfp_customeridname", "msnfp_name"]] \
                  .rename(columns = {"msnfp_customeridname": "Donor"}), \
                  on = ["Donor"]).drop(labels = ["Payment Schedule"], axis = 1).rename(columns = {"msnfp_name": "Payment Schedule"}), \
           df_transaction[~df_transaction["Donor"].isin(df_transaction[df_transaction["Donor"] \
                                                                      .isin(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description]) \
                                                                            [(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description])["msnfp_customeridname"].str.lower() \
                                                                              .isin(df_transaction[df_transaction["Payment Schedule"].isnull()]["Donor"].str.lower()))]["msnfp_customeridname"])] \
                                                        .merge(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description]) \
                                                               [(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description])["msnfp_customeridname"].str.lower() \
                                                                 .isin(df_transaction[df_transaction["Payment Schedule"].isnull()]["Donor"].str.lower()))][["msnfp_customeridname", "msnfp_name"]] \
                                                               .rename(columns = {"msnfp_customeridname": "Donor"}), \
                                                               on = ["Donor"]).drop(labels = ["Payment Schedule"], axis = 1).rename(columns = {"msnfp_name": "Payment Schedule"})["Donor"])]], \
          axis = 0)
df_transaction["Payment Schedule"] = \
np.select([(df_transaction["msnfp_name_x"].notnull()) & (df_transaction["msnfp_name_y"].notnull()) & (df_transaction["msnfp_name_x"]) == (df_transaction["msnfp_name_y"]), \
           (df_transaction["msnfp_name_x"].notnull()) & (df_transaction["msnfp_name_y"].isnull()), \
           (df_transaction["msnfp_name_x"].isnull()) & (df_transaction["msnfp_name_y"].notnull()), \
           (df_transaction["msnfp_name"].notnull())], \
          [df_transaction["msnfp_name_x"], df_transaction["msnfp_name_x"], df_transaction["msnfp_name_y"], df_transaction["msnfp_name"]], \
          default = df_transaction["Payment Schedule"])
df_transaction = df_transaction.drop(labels = ["msnfp_name_x", "msnfp_name_y", "msnfp_name"], axis = 1)

# Get Constituent ID of individuals donors
# While loop to re-establish connection to server in case it fails/errors out until connection is re-established
while True:
    try:
        query_result = \
        cursor.execute("""
                       SELECT DISTINCT(msnfp_constituentnumber), hri_alternativeconstituentnumber,
                              fullname, emailaddress1
                       FROM dbo.contact
                       """).fetchall()
    except pyodbc.Error as error:
        if error.args[0] == "08S01": 
            connection.close()
            connection = connect_to_server(server)
            cursor = connection.cursor()
        try:
            query_result = \
            cursor.execute("""
                           SELECT DISTINCT(msnfp_constituentnumber), hri_alternativeconstituentnumber,
                                  fullname, emailaddress1
                           FROM dbo.contact
                           """).fetchall()
        except: pass
    if query_result != np.nan: break  
df_transaction["join_on_conID"] = \
np.select([(df_transaction["Email"].notnull()) & (df_transaction["Related Constituent"].isnull()), \
           (df_transaction["Email"].isnull()) & (df_transaction["Related Constituent"].notnull()), \
           (df_transaction["Data Entry Reference"].str.contains("Insight") == True)], \
          [(df_transaction["Donor"].str.replace(" ", "") + df_transaction["Email"]).str.lower(), \
           (df_transaction["Donor"].str.replace(" ", "") + df_transaction["Related Constituent"]).str.lower(), \
           (df_transaction["Donor"].str.replace(" ", "") + df_transaction["Related Constituent"]).str.lower()], \
          default = df_transaction["Donor"])
df_SQL_conID = pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description])
df_SQL_conID["join_on_conID"] = \
np.select([(df_SQL_conID["emailaddress1"].notnull()) & (df_SQL_conID["hri_alternativeconstituentnumber"].isnull()), \
           (df_SQL_conID["emailaddress1"].isnull()) & (df_SQL_conID["hri_alternativeconstituentnumber"].notnull()), \
           (df_SQL_conID["emailaddress1"].notnull()) & (df_SQL_conID["hri_alternativeconstituentnumber"].notnull())], \
          [(df_SQL_conID["fullname"].str.replace(" ", "") + df_SQL_conID["emailaddress1"]).str.lower(), \
           (df_SQL_conID["fullname"].str.replace(" ", "") + df_SQL_conID["hri_alternativeconstituentnumber"]).str.lower(), \
           (df_SQL_conID["fullname"].str.replace(" ", "") + df_SQL_conID["hri_alternativeconstituentnumber"]).str.lower()], \
          default = df_SQL_conID["fullname"])
df_SQL_conID = df_SQL_conID[df_SQL_conID["join_on_conID"].isin(df_SQL_conID["join_on_conID"])]
# Import WPG Cons ID from Helena's list
df_HY_WPG_conID = pd.read_excel(f"./Reference Files/HY WPG Cons ID.xlsx")
df_HY_WPG_conID["join_on_conID"] = \
np.select([df_HY_WPG_conID["emailaddress1"].notnull()], \
          [(df_HY_WPG_conID["fullname"].str.replace(" ", "") + df_HY_WPG_conID["emailaddress1"]).str.lower()], \
          default = df_HY_WPG_conID["fullname"])
df_queried_conID = pd.concat([df_SQL_conID, df_HY_WPG_conID], axis = 0)
df_transaction = df_transaction.merge(df_queried_conID, on = ["join_on_conID"], how = "left") \
                               .drop(labels = ["fullname", "emailaddress1"], \
                                     axis = 1)
df_transaction = \
pd.concat([df_transaction[df_transaction["Donor"].isin(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description]) \
                                                       [(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description])["fullname"].str.lower() \
                                                        .isin(df_transaction[df_transaction["msnfp_constituentnumber"].isnull()]["Donor"].str.lower()))]["fullname"])] \
           .merge(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description]) \
                  [(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description])["fullname"].str.lower() \
                    .isin(df_transaction[df_transaction["msnfp_constituentnumber"].isnull()]["Donor"].str.lower()))][["msnfp_constituentnumber", "fullname"]] \
                  .rename(columns = {"fullname": "Donor"}), \
                  on = ["Donor"]), \
           df_transaction[~df_transaction["Donor"].isin(df_transaction[df_transaction["Donor"] \
                                                                      .isin(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description]) \
                                                                            [(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description])["fullname"].str.lower() \
                                                                              .isin(df_transaction[df_transaction["msnfp_constituentnumber"].isnull()]["Donor"].str.lower()))]["fullname"])] \
                                                        .merge(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description]) \
                                                               [(pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description])["fullname"].str.lower() \
                                                                 .isin(df_transaction[df_transaction["msnfp_constituentnumber"].isnull()]["Donor"].str.lower()))][["msnfp_constituentnumber", "fullname"]] \
                                                               .rename(columns = {"fullname": "Donor"}), \
                                                               on = ["Donor"])["Donor"])]], \
          axis = 0)
df_transaction["msnfp_constituentnumber"] = np.select([(df_transaction["msnfp_constituentnumber"].notna()) & (df_transaction["msnfp_constituentnumber_y"].isnull()), \
                                                       (df_transaction["msnfp_constituentnumber"].isnull()) & (df_transaction["msnfp_constituentnumber_y"].notna()), \
                                                       (df_transaction["msnfp_constituentnumber"].isnull()) & (df_transaction["msnfp_constituentnumber_y"].isnull())], \
                                                      [df_transaction["msnfp_constituentnumber"], df_transaction["msnfp_constituentnumber_y"], None])
df_transaction = df_transaction.drop(labels = ["msnfp_constituentnumber_x", "msnfp_constituentnumber_y"], axis = 1)
df_transaction = df_transaction.drop_duplicates(subset = ["Identifier"], ignore_index = True)

# Double-check for payschedID & conID
df_transaction["Payment Schedule"] = \
[("DIGI AU Braintree (SubID: " + df_transaction["join_on_payschedID"][i] + " )") \
 if ("Braintree" in df_transaction["Data Entry Reference"][i]) \
    & (True in list(df_payschedID["msnfp_name"].str.contains(str(df_transaction["join_on_payschedID"][i])))) \
 else None \
 for i in range(len(df_transaction))]
df_transaction["Payment Schedule"] = \
np.select([(df_transaction["Currency"] == "NZD") & (df_transaction["Data Entry Reference"].str.contains("Braintree")), \
           (df_transaction["Currency"] == "GBP") & (df_transaction["Data Entry Reference"].str.contains("Braintree"))], \
          [df_transaction["Payment Schedule"].str.replace("AU", "NZ"), \
           df_transaction["Payment Schedule"].str.replace("AU", "UK")], \
          default = df_transaction["Payment Schedule"])
list_match_prob, list_conID = list(), list()
query_iterator = 0 
query_iterator_limit = len(df_transaction["Donor"]) // index_donor_step
settings = {"link_type": "dedupe_only", \
            "blocking_rules_to_generate_predictions": 
                ["l.msnfp_customeridname = r.msnfp_customeridname", \
                 "l.emailaddress1 = r.emailaddress1", \
                 "l.msnfp_originatingcampaignidname = r.msnfp_originatingcampaignidname", \
                 "l.msnfp_designationidname = r.msnfp_designationidname", \
                 "l.msnfp_appealidname = r.msnfp_appealidname", \
                 "l.msnfp_paymenttypecodename = r.msnfp_paymenttypecodename", \
                 "l.msnfp_transaction_paymentmethodidname = r.msnfp_transaction_paymentmethodidname"], \
            "comparisons": 
                [cl.jaro_at_thresholds("msnfp_customeridname", [0.9, 0.7], term_frequency_adjustments = True), \
                 cl.levenshtein_at_thresholds("emailaddress1"), \
                 cl.jaro_at_thresholds("msnfp_paymenttypecodename", [0.9, 0.7], term_frequency_adjustments = True), \
                 cl.jaro_winkler_at_thresholds("msnfp_originatingcampaignidname", [0.9, 0.7], term_frequency_adjustments = True), \
                 cl.jaro_winkler_at_thresholds("msnfp_designationidname", [0.9, 0.7], term_frequency_adjustments = True)]}
deterministic_rules = ["l.msnfp_originatingcampaignidname = r.msnfp_originatingcampaignidname", \
                       "l.msnfp_designationidname = r.msnfp_designationidname", \
                       "l.msnfp_appealidname = r.msnfp_appealidname", \
                       "l.emailaddress1 = r.emailaddress1"]
df_temp = df_transaction[["Amount", "Book Date", "Donor", "Email", "Originating Campaign", "Primary Designation", \
                          "Payment Schedule", "Appeal", "Data Entry Source", "Package", "Card Brand", \
                          "Payment Type", "Payment Method", "Currency", "msnfp_constituentnumber"]] \
                        .rename(columns = \
                               {"Amount": "msnfp_amount", "Book Date": "msnfp_bookdate", \
                                "Donor": "msnfp_customeridname", "Email": "emailaddress1", \
                                "Originating Campaign": "msnfp_originatingcampaignidname", \
                                "Primary Designation": "msnfp_designationidname", \
                                "Payment Schedule": "msnfp_transaction_paymentscheduleidname", \
                                "Appeal": "msnfp_appealidname", "Data Entry Source": "msnfp_dataentrysourcename", \
                                "Package": "msnfp_packageidname", "Card Brand": "msnfp_ccbrandcodename", \
                                "Payment Type": "msnfp_paymenttypecodename", \
                                "Payment Method": "msnfp_transaction_paymentmethodidname", \
                                "Currency": "transactioncurrencyidname"})
query_prefix = \
"""
SELECT T.msnfp_amount, T.msnfp_bookdate, 
	   T.msnfp_customeridname, C.emailaddress1, 
	   T.msnfp_originatingcampaignidname, 
	   T.msnfp_designationidname, 
	   T.msnfp_transaction_paymentscheduleidname, 
	   T.msnfp_appealidname, T.msnfp_dataentrysourcename, 
	   T.msnfp_packageidname, T.msnfp_ccbrandcodename, 
	   T.msnfp_paymenttypecodename, 
	   T.msnfp_transaction_paymentmethodidname, 
	   T.transactioncurrencyidname, C.msnfp_constituentnumber                           
FROM dbo.msnfp_transaction AS T JOIN dbo.contact AS C 
ON T.msnfp_customerid = C.contactid
WHERE
"""
while True:
    query = query_prefix
    for donor in range(len(df_transaction["Donor"][index_donor_start : index_donor_end])):
        if df_transaction["Email"][index_donor_start : index_donor_end].iloc[donor] is not np.nan:
            if "'" in df_transaction["Donor"][index_donor_start : index_donor_end].iloc[donor]:
                query += " T.msnfp_customeridname LIKE '%" \
                         + df_transaction["Donor"][index_donor_start : index_donor_end].iloc[donor].replace("'", "''") + "%' OR" \
                         + " C.emailaddress1 = '" + df_transaction["Email"][index_donor_start : index_donor_end].iloc[donor].lower() + "' OR"
            if "'" not in df_transaction["Donor"][index_donor_start : index_donor_end].iloc[donor]:
                query += " T.msnfp_customeridname LIKE '%" \
                         + df_transaction["Donor"][index_donor_start : index_donor_end].iloc[donor] + "%' OR " \
                         + "C.emailaddress1 = '" + df_transaction["Email"][index_donor_start : index_donor_end].iloc[donor].lower() + "' OR"   
        if df_transaction["Email"][index_donor_start : index_donor_end].iloc[donor] is np.nan:
            if "'" in df_transaction["Donor"][index_donor_start : index_donor_end].iloc[donor]:
                query += " T.msnfp_customeridname LIKE '%" \
                         + df_transaction["Donor"][index_donor_start : index_donor_end].iloc[donor].replace("'", "''") + "%' OR"
            if "'" not in df_transaction["Donor"][index_donor_start : index_donor_end].iloc[donor]:
                query += " T.msnfp_customeridname LIKE '%" \
                         + df_transaction["Donor"][index_donor_start : index_donor_end].iloc[donor] + "%' OR "
    query = query[:-3]
    try:
        print("Query in progress... (main)")
        query_result = pd.DataFrame.from_records(cursor.execute(query).fetchall(), \
                                                 columns = [i[0] for i in cursor.description])
        if index_donor_end > len(df_transaction["Donor"]): index_donor_end = len(df_transaction["Donor"])
        print(f'Last donor index: {index_donor_end} out of {len(df_transaction["Donor"])}')
        df_temp = pd.concat([df_temp, query_result], axis = 0)
        index_donor_start += index_donor_step
        index_donor_end += index_donor_step
        query_iterator += 1
    except pyodbc.Error as error:
        if error.args[0] == "08S01":
            print("Reconnecting...")
            connection.close()
            connection = connect_to_server(server)
            cursor = connection.cursor()
        try:
            print("Query in progress... (except)")
            query_result = pd.DataFrame.from_records(cursor.execute(query).fetchall(), \
                                                     columns = [i[0] for i in cursor.description])
            if index_donor_end > len(df_transaction["Donor"]): index_donor_end = len(df_transaction["Donor"])
            print(f'Last donor index: {index_donor_end} out of {len(df_transaction["Donor"])}')
            df_temp = pd.concat([df_temp, query_result], axis = 0)
            index_donor_start += index_donor_step
            index_donor_end += index_donor_step
            query_iterator += 1
        except: pass
    if (query_result is not None) & (query_iterator > query_iterator_limit): break
print("Query complete!")
for i in range(len(df_transaction)):
    df_single_donor = df_temp[df_temp["msnfp_customeridname"].str.lower().str.contains(df_transaction["Donor"][i].lower())] \
                             .sort_values(by = ["msnfp_bookdate"], ascending = False)
    if len(df_single_donor) == 1:
        try:
            df_single_donor = df_temp[df_temp["emailaddress1"].str.contains(df_transaction["Email"][i], na = False)] \
                                     .sort_values(by = ["msnfp_bookdate"], ascending = False)     
        except:
            pass
    df_single_donor["unique_id"] = [f"placeholder_id_{i}" for i in range(len(df_single_donor))]
    if len(df_single_donor) == 1:
        list_match_prob.append(None)
        list_conID.append(None)
    if len(df_single_donor) > 1:
        linker = DuckDBLinker(df_single_donor, settings, set_up_basic_logging = False)
        linker.estimate_probability_two_random_records_match(deterministic_rules, recall = 1)
        linker.estimate_u_using_random_sampling(max_pairs = 1e6)
        try:
            linkage = linker.predict(threshold_match_probability = None)
            df_linkage = linkage.as_pandas_dataframe()
            df_linkage = df_linkage[df_linkage["unique_id_l"] == "placeholder_id_0"].sort_values(by = ["match_probability"], \
                                                                                                 ascending = False)
            if len(df_linkage[~(df_linkage["msnfp_customeridname_l"].str.replace(" ", "").str.lower() \
                                != df_linkage["msnfp_customeridname_r"].str.replace(" ", "").str.lower())]) > 0:
                df_linkage = df_linkage[~(df_linkage["msnfp_customeridname_l"].str.replace(" ", "").str.lower() \
                                          != df_linkage["msnfp_customeridname_r"].str.replace(" ", "").str.lower())]    
            if len(df_linkage[~(df_linkage["msnfp_customeridname_l"].str.replace(" ", "").str.lower() \
                                != df_linkage["msnfp_customeridname_r"].str.replace(" ", "").str.lower())]) == 0:
                df_linkage = df_linkage[~(df_linkage["emailaddress1_l"].str.replace(" ", "").str.lower() \
                                          != df_linkage["emailaddress1_r"].str.replace(" ", "").str.lower())]  
            list_match_prob.append(df_linkage.iloc[0, :]["match_probability"])
            list_conID.append(df_single_donor[df_single_donor["unique_id"] \
                              == df_linkage.iloc[0, :]["unique_id_r"]]["msnfp_constituentnumber"].iloc[0])
        except:
            list_match_prob.append(None)
            list_conID.append(None)
for i in list(df_transaction[df_transaction["Donor"].str.contains("Anonymous")].index):
    list_conID.insert(i, None) ; list_match_prob.insert(i, float(1))
df_transaction["msnfp_constituentnumber"] = list_conID
df_transaction["match_prob"] = list_match_prob
df_transaction["msnfp_constituentnumber"] = \
np.select([(df_transaction["Donor"] == "Anonymous AU"), \
           (df_transaction["Donor"] == "Anonymous NZ"), \
           (df_transaction["Donor"] == "Anonymous UK")], \
          ["151334", "150888", "143900"], \
          default = df_transaction["msnfp_constituentnumber"])
for i in list(df_transaction[(df_transaction["Payment Schedule"].str.contains("Braintree", na = False)) \
                             & (df_transaction["msnfp_constituentnumber"].isnull())].index):
    query_prefix = \
    """
    SELECT C.msnfp_constituentnumber                          
    FROM dbo.msnfp_transaction AS T JOIN dbo.contact AS C ON T.msnfp_customerid = C.contactid
    WHERE
    """
    query = query_prefix + "T.msnfp_transaction_paymentscheduleidname LIKE '%" + df_transaction["join_on_payschedID"][i] + "%'"
    try:
        print("Query in progress... (main)")
        query_result = pd.DataFrame.from_records(cursor.execute(query).fetchall(), \
                                                 columns = [i[0] for i in cursor.description])
    except pyodbc.Error as error:
        if error.args[0] == "08S01":
            print("Reconnecting...")
            connection.close()
            connection = connect_to_server(server)
            cursor = connection.cursor()
        try:
            print("Query in progress... (except)")
            query_result = pd.DataFrame.from_records(cursor.execute(query).fetchall(), \
                                                     columns = [i[0] for i in cursor.description])
        except: pass
    if len(query_result) > 0:
        df_transaction["msnfp_constituentnumber"][i] = str(query_result["msnfp_constituentnumber"][0])
        df_transaction["match_prob"][i] = float(1)
    if len(query_result) == 0:
        df_transaction["msnfp_constituentnumber"][i] = None
        df_transaction["match_prob"][i] = None
for i in list(df_transaction[(df_transaction["Data Entry Reference"].str.contains("Insight"))].index):
    if df_transaction["Related Constituent"][i] in list(df_SQL_conID[df_SQL_conID["hri_alternativeconstituentnumber"].notnull()]["hri_alternativeconstituentnumber"]):
        df_transaction["msnfp_constituentnumber"][i] = \
        df_SQL_conID[df_SQL_conID["hri_alternativeconstituentnumber"] == df_transaction["Related Constituent"][i]]["msnfp_constituentnumber"].values[0]
        df_transaction["match_prob"][i] = float(1)
    if df_transaction["Related Constituent"][i] not in list(df_SQL_conID[df_SQL_conID["hri_alternativeconstituentnumber"].notnull()]["hri_alternativeconstituentnumber"]):
        df_transaction["msnfp_constituentnumber"][i] = None
        df_transaction["match_prob"][i] = None

# Get Constituent ID of organizations donors
# While loop to re-establish connection to server in case it fails/errors out until connection is re-established
while True:
    try:
        query_result = \
        cursor.execute("""
                       SELECT DISTINCT(acc.msnfp_constituentnumber),
                              acc.name
                       FROM dbo.account AS acc
                       """).fetchall()
    except pyodbc.Error as error:
        if error.args[0] == "08S01": 
            connection.close()
            connection = connect_to_server(server)
            cursor = connection.cursor()
        try:
            query_result = \
            cursor.execute("""
                           SELECT DISTINCT(acc.accountnumber),
                                  acc.name
                           FROM dbo.account AS acc
                           """).fetchall()
        except: pass
    if query_result != np.nan: break 
df_acc_conID = pd.DataFrame.from_records(query_result, columns = [i[0] for i in cursor.description])
for i in list(df_transaction[(df_transaction["Ref"].str.contains("Matched", na = False)) & (df_transaction["Donor"].str.contains("Anonymous"))].index):
    df_transaction["msnfp_constituentnumber"][i] = df_acc_conID[df_acc_conID["name"] == df_transaction["Organization Name"][i]]["accountnumber"]

# Handle incorrectly detected dates, if any
df_transaction["Book Date"] = [pd.to_datetime(pd.to_datetime(i).strftime("%Y-%d-%m")) \
                               if pd.to_datetime(i) > datetime.now() \
                               else pd.to_datetime(i) \
                               for i in df_transaction["Book Date"]]
# Assert data types & mapping
df_transaction["Amount"] = df_transaction["Amount"].astype("float")
df_transaction["Currency"] = df_transaction["Currency"].map({"AUD": "Australian Dollar", \
                                                             "NZD": "New Zealand Dollar", \
                                                             "GBP": "UK Pound", \
                                                             "CAD": "Canadian Dollar"})
    
# Assign a flagger for each donor
df_transaction["donor_flagger"] = \
np.select([(df_transaction["Data Entry Reference"].str.contains("Braintree")) & (df_transaction["Payment Schedule"].notna()) \
           & (df_transaction["join_on_payschedID"].notna()) & (df_transaction["msnfp_constituentnumber"].notna()) \
           & (df_transaction["match_prob"] < match_threshold), \
           (df_transaction["Data Entry Reference"].str.contains("Braintree")) & (df_transaction["Payment Schedule"].isnull()) \
           & (df_transaction["join_on_payschedID"].isnull()) & (df_transaction["msnfp_constituentnumber"].notna()) \
           & (df_transaction["match_prob"] < match_threshold), \
           (df_transaction["Data Entry Reference"].str.contains("Braintree")) & (df_transaction["Payment Schedule"].isnull()) \
           & (df_transaction["join_on_payschedID"].notna()) & (df_transaction["msnfp_constituentnumber"].isnull()) \
           & (df_transaction["match_prob"].isnull()), \
           (df_transaction["Data Entry Reference"].str.contains("Braintree")) & (df_transaction["Payment Schedule"].isnull()) \
           & (df_transaction["join_on_payschedID"].notna()) & (df_transaction["msnfp_constituentnumber"].notna()) \
           & (df_transaction["match_prob"] < match_threshold), \
           (df_transaction["Data Entry Reference"].str.contains("Braintree")) & (df_transaction["Payment Schedule"].isnull()) \
           & (df_transaction["join_on_payschedID"].notna()) & (df_transaction["msnfp_constituentnumber"].notna()) \
           & (df_transaction["match_prob"] >= match_threshold), \
           (df_transaction["Data Entry Reference"].str.contains("Braintree")) & (df_transaction["Payment Schedule"].notna()) \
           & (df_transaction["msnfp_constituentnumber"].isnull()) & (df_transaction["match_prob"].isnull()), \
           (df_transaction["Data Entry Reference"].str.contains("Braintree") == False) \
           & (df_transaction["msnfp_constituentnumber"].notna()) & (df_transaction["match_prob"] < match_threshold), \
           (df_transaction["Data Entry Reference"].str.contains("Braintree") == False) \
           & (df_transaction["msnfp_constituentnumber"].isnull()) & (df_transaction["match_prob"].isnull())], \
          ["check_conID", "check_conID", "new_conID_new_payschedID", "check_conID_new_payschedID", "new_payschedID", "new_conID", \
           "check_conID", "new_conID"], \
          default = "all_good")

# Export draft df Transaction template
df_transaction.to_excel(f"./{recon_month} {str(start_date)[:4]} Recon/Templates/Template - Transaction - Draft.xlsx", \
                        index = False)

# Pop out dialog box to confirm draft df Transaction template is double-checked
checker = simpledialog.askstring(title = "Type `done` once draft df Transcation template is double-checeked", \
                                 prompt = f'Double-check:{" " * 140}')

# Export final df Transaction template to Excel file
df_transaction_draft = pd.read_excel(f"./{recon_month} {str(start_date)[:4]} Recon/Templates/Template - Transaction - Draft.xlsx")
df_transaction["Received Date"] = df_transaction["Book Date"]
df_transaction["Phone"] = np.select([df_transaction["Phone"].notna()], ["0" + df_transaction["Phone"].astype("Int64").astype("str")], \
                                    default = df_transaction["Phone"])
df_transaction["Phone"] = df_transaction["Phone"].astype("str")
df_transaction["ZIP/Postal Code"] = pd.to_numeric(df_transaction["ZIP/Postal Code"], errors = "coerce")
df_transaction["ZIP/Postal Code"] = np.select([df_transaction["ZIP/Postal Code"].astype("Int64").astype("str").str.len() < 4, df_transaction["ZIP/Postal Code"].isnull()], \
                                              ["0" + df_transaction["ZIP/Postal Code"].astype("Int64").astype("str"), None], \
                                              default = df_transaction["ZIP/Postal Code"].astype("Int64").astype("str"))
df_transaction["ZIP/Postal Code"] = df_transaction["ZIP/Postal Code"].astype("str")
df_transaction = df_transaction_draft[list_df_transaction_columns]
df_transaction.to_excel(f"./{recon_month} {str(start_date)[:4]} Recon/Templates/Template - Transaction - {recon_month} {str(start_date)[:4]}.xlsx", \
                        index = False)

# Final df Contact template
df_temp = df_transaction_draft[["Donor", "Email", "Country/Region", "Currency", \
                                "Status", "Status Reason", "First Name", "Last Name", "Phone", \
                                "State/Province", "Street 1", "City", "ZIP/Postal Code", \
                                "Organization Name", "Related Constituent", \
                                "msnfp_constituentnumber", "donor_flagger"]] \
                              .rename(columns = {"Donor": "Full Name", "Phone": "Address 1: Phone", \
                                                 "State/Province": "Address 1: State/Province", "Street 1": "Address 1: Street 1", \
                                                 "City": "Address 1: City", "ZIP/Postal Code": "Address 1: ZIP/Postal Code", \
                                                 "Country/Region": "Address 1: Country/Region", "Organization Name": "Company", \
                                                 "Related Constituent": "Alternative Constituent Number"})
df_temp["Address 1: Phone"] = np.select([df_temp["Address 1: Phone"].notna()], ["0" + df_temp["Address 1: Phone"].astype("Int64").astype("str")], \
                                        default = df_temp["Address 1: Phone"])
df_temp["Address 1: ZIP/Postal Code"] = np.select([df_temp["Address 1: ZIP/Postal Code"].astype("Int64").astype("str").str.len() < 4, \
                                                   df_temp["Address 1: ZIP/Postal Code"].isnull()], \
                                                  ["0" + df_temp["Address 1: ZIP/Postal Code"].astype("Int64").astype("str"), None], \
                                                  default = df_temp["Address 1: ZIP/Postal Code"].astype("Int64").astype("str"))
df_temp = df_temp[(df_temp["msnfp_constituentnumber"].isnull()) \
                  & ((df_temp["donor_flagger"] == "new_conID") | (df_temp["donor_flagger"] == "new_conID_new_payschedID"))].reset_index(drop = True)
last_conID_index = \
int(df_SQL_conID[df_SQL_conID["msnfp_constituentnumber"].str.contains("CON")].sort_values(by = ["msnfp_constituentnumber"], ascending = False) \
                .iloc[0]["msnfp_constituentnumber"].split("-")[-1])
# While loop to ensure unique conID generation
while True:
    df_temp["Constituent Number"] = "CON-" + pd.Series([last_conID_index + i for i in range(1, len(df_temp) + 1)]).astype("str")
    if (len(df_SQL_conID["msnfp_constituentnumber"].unique()) + len(df_temp["Constituent Number"].unique()) \
        == len(df_SQL_conID["msnfp_constituentnumber"].unique()) + len(df_temp["Full Name"].unique())):
        break
    continue 
df_contact = pd.concat([pd.read_excel("./Reference Files/Contact Template Official.xlsx"), df_temp], \
                       axis = 0).iloc[1:, :].drop(labels = ["msnfp_constituentnumber", "donor_flagger"], axis = 1)
# Export final df Contact template to Excel file
df_contact.to_excel(f"./{recon_month} {str(start_date)[:4]} Recon/Templates/Template - Contact - {recon_month} {str(start_date)[:4]}.xlsx", \
                    index = False)

# Final df Account template
df_temp = df_transaction_draft[["Donor", "Organization Name", "Email", "Country/Region", "Currency", \
                                "Status", "Status Reason", "First Name", "Last Name", "Phone", \
                                "State/Province", "Street 1", "City", "ZIP/Postal Code", "Ref", \
                                "msnfp_constituentnumber", "donor_flagger"]] \
                              .rename(columns = {"Organization Name": "Account Name", "Phone": "Main Phone", \
                                                 "State/Province": "Address 1: State/Province", "Street 1": "Address 1: Street 1", \
                                                 "City": "Address 1: City", "ZIP/Postal Code": "Address 1: ZIP/Postal Code", \
                                                 "Country/Region": "Address 1: Country/Region"})
df_temp = df_temp[(df_temp["msnfp_constituentnumber"].isnull()) & (df_temp["Ref"].str.contains("Matched", na = False)) \
                  & (df_temp["Donor"].str.contains("Anonymous")) \
                  & ((df_temp["donor_flagger"] == "new_conID") | (df_temp["donor_flagger"] == "new_payschedID_conID"))].reset_index(drop = True)
last_conID_index = \
int(df_acc_conID[df_acc_conID["msnfp_constituentnumber"].str.contains("ACC")].sort_values(by = ["msnfp_constituentnumber"], ascending = False) \
                .iloc[0]["msnfp_constituentnumber"].split("-")[-1])
df_temp["Constituent Number"] = "ACC-" + pd.Series([last_conID_index + i for i in range(1, len(df_temp) + 1)]).astype("str")
df_account = pd.concat([pd.read_excel("./Reference Files/Account Template Official.xlsx"), df_temp], \
                       axis = 0).iloc[1:, :].drop(labels = ["Donor", "Ref", "msnfp_constituentnumber", "donor_flagger"], axis = 1)
# Export final df Account template to Excel file
df_account.to_excel(f"./{recon_month} {str(start_date)[:4]} Recon/Templates/Template - Account - {recon_month} {str(start_date)[:4]}.xlsx", \
                    index = False)

# Final df Payment Schedule template
df_temp = df_transaction_draft[["Donor", "Payment Type", "Originating Campaign", "Appeal", "Organization Name", \
                                "Package", "Book Date", "Primary Designation", "Status Reason", \
                                "Data Entry Source", "Payment Method", "First Name", "Last Name", \
                                "State/Province", "Country/Region", "Amount", "Anonymous", "Subscription ID", \
                                "Email", "Card Brand", "Transaction ID", "Payment Schedule", "Currency", "donor_flagger"]] \
                              .rename(columns = {"Payment Schedule": "Identifier", \
                                                 "Amount": "Amount (Tax Deductible)", \
                                                 "Data Entry Source": "Source"})
df_temp = df_temp[(df_temp["Identifier"].isnull()) \
                  & ((df_temp["donor_flagger"] == "new_payschedID") | (df_temp["donor_flagger"] == "new_conID_new_payschedID") \
                     | (df_temp["donor_flagger"] == "check_conID_new_payschedID"))].reset_index(drop = True)
df_temp["Status Reason"] = "In Progress"
# While loop to ensure unique payschedID generation
while True:
    df_temp["Identifier"] = "DIGI AU Braintree (SubID: " + df_temp["Subscription ID"].astype("str") + " )"
    if (len(df_payschedID["msnfp_name"].unique()) + len(df_temp["Identifier"].unique()) \
        == len(df_payschedID["msnfp_name"].unique()) + len(df_temp["Donor"].unique())):
        break
    continue
df_temp["Identifier"] = \
np.select([df_temp["Currency"] == "New Zealand Dollar", df_temp["Currency"] == "UK Pound"], \
          [df_temp["Identifier"].str.replace("AU", "NZ"), df_temp["Identifier"].str.replace("AU", "UK")], \
          default = df_temp["Identifier"])
df_paysched = pd.concat([pd.read_excel("./Reference Files/Payment Schedule Template Official.xlsx"), df_temp], \
                        axis = 0).iloc[1:, :].drop(labels = ["donor_flagger"], axis = 1)
# Export final df Payment Schedule template to Excel file
df_paysched.to_excel(f"./{recon_month} {str(start_date)[:4]} Recon/Templates/Template - Payment Schedule - {recon_month} {str(start_date)[:4]}.xlsx", \
                     index = False)